In [1]:
import openai
import os
import requests
from pathlib import Path
from urllib.parse import urlparse

In [2]:
pdf_url = 'https://cdn.openai.com/research-covers/language-unsupervised/language_understanding_paper.pdf'
r = requests.get(pdf_url)
if r.status_code != 200:
    raise ValueError(
        'Check the URL of your file; returned status code: %s' % r.status_code
    )

parsed_url = urlparse(pdf_url)
file_name = os.path.basename(parsed_url.path)

pdf_file_dir = Path('pdf_files')
pdf_file_dir.mkdir(exist_ok=True)
file_path = pdf_file_dir / file_name

with open(file_path, 'wb') as f:
    f.write(r.content)

print('Saved file to %s' % file_path)

from pdfminer.high_level import extract_text
text = extract_text(str(file_path))

Saved file to pdf_files/language_understanding_paper.pdf


In [3]:
# 文字か数字かを判定. 数字の場合はTrueを返す
def is_digit_or_uppercase_word(element):
    if element.isdigit() or element[0].isupper():
        return True
    return False
chunk_idx = []
chunk_key_names = []
for idx, chunk in enumerate(text.split('\n\n')):
    chunk_elements = chunk.split()
    chunk_judgement = [is_digit_or_uppercase_word(element) for element in chunk_elements]
    if all(chunk_judgement):
        if len(chunk) > 3:
            # print(chunk)
            # print('chunk_idx:',idx)
            chunk_idx.append(idx)
            chunk_key_names.append(chunk)


In [4]:
chunk_dict = dict()

for idx, chunk_key_name in enumerate(chunk_key_names):
    if idx == len(chunk_key_names)-1:
        chunk_dict[chunk_key_name] = (chunk_idx[idx], len(text.split('\n\n')))
    else:
        chunk_dict[chunk_key_name] = (chunk_idx[idx], chunk_idx[idx+1])

In [5]:
chunk_dict

{'Abstract': (5, 8),
 'Introduction': (8, 16),
 '2 Related Work': (16, 24),
 '3 Framework': (24, 57),
 '4 Experiments': (57, 62),
 'Task': (62, 63),
 'Datasets': (63, 76),
 'Method': (177, 178),
 'MNLI-m MNLI-mm SNLI': (77, 78),
 'SciTail QNLI RTE': (78, 106),
 'Story Cloze RACE-m RACE-h RACE': (107, 128),
 'Classiﬁcation': (129, 130),
 'Semantic Similarity': (130, 131),
 'GLUE': (131, 171),
 '5 Analysis': (171, 177),
 'Avg. Score': (178, 209),
 '6 Conclusion': (209, 211),
 'References': (211, 332)}

In [6]:
section = dict()
for chunk_key_name in chunk_dict.keys():
    sentences = text.split('\n\n')[chunk_dict[chunk_key_name][0]+1 :chunk_dict[chunk_key_name][1]]
    sentences = ''.join(sentences)
    sentences = sentences.replace('\n','')
    section[chunk_key_name] = sentences

**temperature**<br>
あくまで、LLMは次に来る単語を予測するモデル.<br>
temperatureを変更することで、予測される単語の多様性を変更できる.<br>

temperatureが0だと、常に一番高い確率のものを選択し続ける. 1に近づくにつれリスクを取り、確率の低い単語も選択し、多様性が増す.<br>

In [14]:
# temperatureは確信度を表すパラメータ
# 0に近いほど確信度が高くなる(回答が固定される)
# 1に近いほど確信度が低くなる(回答がランダムになる)
openai.api_key = os.getenv("OPENAI_API_KEY")
MODEL = "gpt-3.5-turbo"
response = openai.ChatCompletion.create(
    model=MODEL,
    messages=[
        {'role': "system", 'content': 'You are a helpful assistant'},
        {'role': "user", 'content': 'Knock knock'},
        {'role': 'assistant', 'content': 'Who\'s there?'},
        {'role': "user", 'content': 'Orange'}
    ],
    temperature=0,
)

response['choices'][0]['message']['content']

'Orange who?'